$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}$


# Localização de zeros de funções

## $ \S 1 $ Introdução

Um ponto $ \zeta $ do domínio de uma função $ f $ de uma variável é dito um **zero** de $ f $ caso
$$ f(\zeta) = 0 .$$
Nesta situação também dizemos que $ \zeta $ é uma **raiz** da equação acima.

📝 '$ \zeta $' é a sexta letra do alfabeto grego, denominada *zeta*.

A busca por zeros de uma função é um dos problemas mais freqüentes em Ciência e Engenharia. Embora a determinação *analítica* dos zeros seja difícil mesmo nos casos mais simples, por exemplo em que a função é um polinômio de grau $ > 2 $, existem métodos numéricos gerais que nos permitem encontrar zeros de funções contínuas arbitrárias com alta precisão e baixo uso de recursos computacionais.

📝 Observe que encontrar uma raiz de uma equação qualquer em uma variável, digamos, da forma 
$$
g(x) = h(x),
$$
é equivalente à tarefa de se encontrar um zero da função $ f(x) = g(x) - h(x) $.

Ainda que a maioria das funções estudadas em Cálculo possam ser estendidas a funções de uma variável complexa, estaremos interessados aqui em encontrar apenas os zeros *reais* de uma *função real contínua de uma variável real*. Contudo, algumas das técnicas que estudaremos, em especial o método de Newton, também podem ser aplicadas a funções complexas.

O sistema de ponto flutuante empregado em computação impõe restrições inevitáveis à precisão do cálculo dos valores de uma função. Por isto na prática não podemos esperar encontrar um ponto $ \zeta $ onde a função valha exatamente $ 0 $, como ilustrado no exemplo abaixo. Em vez disto, buscamos encontrar um *intervalo* suficientemente pequeno onde a função troca de sinal.

**Problema 1:** Considere os polinômios
$$ p(x) = 2x - 0.2 \qquad \text{e} \qquad q(x) = 3 x - 0.3 $$
Note que $ a = \frac{1}{10} $ é um zero tanto de $ p $ quanto de $ q $.

(a) Usando `lambda`, defina $ p(x) $ e $ q(x) $ como funções em Python.

(b) Agora defina um procedimento `checa_zero(f, x)` que, dados uma função $ f $ e um número $ x $ em seu domínio, retorna `True` se `f(x) == 0` e `False` caso contrário. Qual é o resultado da aplicação deste procedimento aos pares $ (p, a) $ e $ (q, a) $? Como você explica esta discrepância?

(c) Como poderíamos modificar a definição de `checa_zero` de modo que ela acuse que $ q $ possui um zero em $ a $? Quais as desvantagens da sua proposta?

*Solução:*

In [77]:
round(2 * 0.1, 26)

0.2

## $ \S 2 $ Localização de raízes

### $ 2.1 $ O Teorema do Valor Intermediário

**Teorema do Valor Intermediário:** Seja $ f \colon [a, b] \to \mathbb R $ uma função _contínua_ definida num intervalo $ [a, b] $ _limitado e fechado_. Então $ f $ assume em $ [a, b] $ todos os valores possíveis entre $ f(a) $ e $ f(b) $.

**Demonstração informal:** Suponha por concretude que $ f(a) < 0 $ e $ f(b) > 0 $. Imaginando o gráfico da função contínua $ f $, isto significa que em $ a $ ele está abaixo do eixo-$x$ e em $ b $ acima. Queremos mostrar que em algum momento ele cruza o eixo-$x$.

Considere o conjunto $ N $ (de negativo) definido por
$$
N = \set{x \in [a,b]}{f(x) < 0}.
$$
Então $ N $ é não-vazio porque contém $ a $. Além disto, $ b $ é cota superior para $ N $. Seja $ c \in [a, b] $ a *menor* cota superior possível para $ N $, i.e.:
1. $ x \le c $ para todo $ x \in N $ (pois exigimos que $ c $ fosse cota superior);
2. Se $ \eps > 0 $, então $ x - \eps $ não é cota superior para $ N $ (pois exigimos que $ c $ fosse a *menor* dentre as cotas superiores).

Então há apenas três opções:
* $ f(c) < 0 $, e por continuidade $ f(x) < 0 $ para todo $ x > c $ suficientemente próximo de $ c $, contradizendo o fato que $ c $ era cota superior de $ N $; ou
* $ f(c) > 0 $, e por continuidade $ f(x) > 0 $ para todo $ x < c $ suficientemente próximo de $ c $, contradizendo o fato que $ c $ era a *menor* cota superior de $ N $; ou
* $ f(c) = 0 $, o que estabelece a conclusão desejada.
<div style="text-align: right">$ \blacksquare $ </div>

O resultado abaixo segue imediatamente do TVI. Apesar de simples, ele é a base dos vários métodos que estudaremos para obtenção de raízes de equações.

**Corolário (encaixotamento de raízes):** Seja $ f \colon [a, b] \to \mathbb R $ uma função contínua. Se
$$ f(a)f(b) < 0\ , $$
então $ f $ possui pelo menos um zero em $ [a, b] $.

Informalmente, se os sinais dos valores de $ f $ em dois pontos  são opostos, então entre eles deve existir algum zero.

**Problema 2:** Mostre que:

(a) Existe um número $ a > 1 $ tal que $ a^a = 23 $.

(b) Este número é único. *Dica:* Considere a função $ f(x) = x^x $ para $ x > 1 $ e calcule sua derivada.

*Solução:*

**Problema 3:** Seja $ n $ um inteiro qualquer.

(a) Mostre analiticamente ou com ajuda do computador que a função tangente assume valores de sinais opostos em
$$
a_n = n\pi + \frac{\pi}{4} \quad \text{e} \quad b_n = n \pi + \frac{3\pi}{4}.
$$

(b) Existe um zero entre $ \frac{\pi}{4} $ e $ \frac{3\pi}{4} $? Justifique, em vista do Corolária acima. *Dica:* Quais são os pontos de descontinuidade da função tangente?

(c) Mais geralmente, existe um zero $ c_n $ no intervalo $ (a_n,b_n) $? E no intervalo $ \big(b_n, a_{n+1}\big) $?

*Solução:*

### $ 2. 2 $ Descrição do procedimento para localização de uma raiz

Todos os métodos para localização de zeros que estudaremos são *iterativos*. Partindo de uma estimativa inicial para um zero $ \zeta $, a cada passo utilizamos a estimativa anterior para obter uma aproximação mais refinada para $ \zeta $, até que esta seja julgada boa o suficiente.

Os *critérios de parada* mais comuns são:
1. A distância entre o valor exato e a aproximação atual é menor que um $ \eps > 0 $ escolhido previamente.
2. O valor da função na aproximação atual é menor que $ \eps $ em valor absoluto.
3. O número de iterações excede uma cota $ N$ pré-fixada.


Qualquer dos métodos para obtenção de zeros que estudaremos requer como passo preliminar o **encaixotamento** de uma raiz, ou seja, a determinação de um intervalo $ [a, b] $ onde $ f $ troca de sinal. A escolha do intervalo inicial é crucial: para algumas escolhas o método em questão pode convergir muito lentamente ou até falhar.

Para encaixotar um zero de uma função, as três opções mais comuns são:
* Usar a a teoria subjacente para advinhar a sua localização aproximada, no caso em que a função provém de um modelo da Física ou Engenharia;
* Esboçar o gráfico da função e estimar visualmente um subintervalo onde ele cruza o eixo-$x$;
* Usar uma busca sistemática, avaliando o sinal da função em pontos sucessivos, para localizar um subintervalo onde a função troca de sinal.

Destes três métodos, apenas o terceiro é rígido o suficiente para ser programado com facilidade, o que não quer dizer que os outros dois devam ser utilizados.

📝 **Isolar** um zero significa encontrar um intervalo que o contém em seu interior mas que não contém qualquer outro zero. Se um intervalo contém mais de um zero, em geral não há como controlar para qual deles um método iterativo convergirá, por isto sempre que possível é desejável isolar um zero, não somente encaixotá-lo.

## $ \S 3 $ Busca incremental para encaixotamento de raízes

In [82]:
def busca_incremnetal(f, a: float, b: float, h: float) -> (float, float):
    """
    Começando com x_1 = a e x_2 = a + h e com incrementos
    de h, retorna o primeiro par de pontos consecutivos onde
    f assume sinais opostos.
    """
    from numpy import sign
    
    x_1 = a
    x_2 = a + h
    f_1 = f(x_1)
    f_2 = f(x_2)
    
    while sign(f_1) == sign(f_2):
        if x_1 >= b:
            return None, None
        x_1 = x_2
        f_1 = f_2
        x_2 += h
        f_2 = f(x_2)
    else:
        return x_1, x_2

⚠️ Por menor que seja o valor do incremento $ h $, não há como garantir *a priori* que o intervalo resultante contenha um *único* zero.

**Problema 4:** O polinômio $ x^3 - 10x^2 + 5 $ possui um zero entre $ 0 $ e $ 1 $. Encaixote este zero dentro de um intervalo de comprimento no máximo $ 10^{-3} $.

## $ \S 4 $ Busca incremental adaptativa